In [1]:
#Importing dependencies
from pprint import pprint
import pandas as pd
import json
import numpy as np

In [2]:
#Loading the U.S. Inflation data 
infl_raw_df = pd.read_csv('Resources/US_CPI.csv')
infl_df = infl_raw_df[['Yearmon','CPI']].rename(columns={'Yearmon':'Date',\
                                                        'CPI':'Consumer_Price_Index'})
#infl_df = infl_df.reset_index(drop=True)
infl_df.head()
#infl_df.sort_values('Date', ascending=True)
#infl_df['Date']=pd.to_datetime(infl_df['Date'], origin='unix', unit='ms').dt.date

,Date,Consumer_Price_Index
0,01-01-1913,9.8
1,01-02-1913,9.8
2,01-03-1913,9.8
3,01-04-1913,9.8
4,01-05-1913,9.7


In [3]:
#Loading the U.S. Gas Price data 
gas_raw_df = pd.read_csv('Resources/PET_PRI_GND_DCUS_NUS_W.csv')
gas_df = gas_raw_df[['Date','A1','A2','A3','D1']].rename(columns={'Date':'Date',\
                                                                  'A1':'All_Grades_Prices_avg',\
                                                                  'A2':'All_Grades_Conventional_Prices_avg',\
                                                                  'A3':'All_Grades_Reformulated_Prices_avg',\
                                                                  'D1':'No2_Diesel_Prices'})
gas_df.head()


,Date,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices
0,01/02/1995,1.127,1.104,1.231,1.104
1,01/09/1995,1.134,1.111,1.232,1.102
2,01/16/1995,1.126,1.102,1.231,1.100
3,01/23/1995,1.132,1.110,1.226,1.095
4,01/30/1995,1.131,1.109,1.221,1.090


In [4]:
#Changing the Date format for Gas dataframe
gas_df['Date'] = pd.to_datetime(gas_df["Date"])
print(gas_df)
print(gas_df.dtypes)

           Date  All_Grades_Prices_avg  All_Grades_Conventional_Prices_avg  \
0    1995-01-02                  1.127                               1.104   
1    1995-01-09                  1.134                               1.111   
2    1995-01-16                  1.126                               1.102   
3    1995-01-23                  1.132                               1.110   
4    1995-01-30                  1.131                               1.109   
...         ...                    ...                                 ...   
1356 2020-12-28                  2.330                               2.225   
1357 2021-01-04                  2.336                               2.227   
1358 2021-01-11                  2.403                               2.298   
1359 2021-01-18                  2.464                               2.351   
1360 2021-01-25                  2.478                               2.363   

      All_Grades_Reformulated_Prices_avg  No2_Diesel_Prices  
0

In [5]:
#Changing the Date format for inflation dataframe
infl_df['Date'] = pd.to_datetime(infl_df["Date"])
print(infl_df)
print(infl_df.dtypes)

           Date  Consumer_Price_Index
0    1913-01-01                 9.800
1    1913-01-02                 9.800
2    1913-01-03                 9.800
3    1913-01-04                 9.800
4    1913-01-05                 9.700
...         ...                   ...
1298 2021-01-03               264.877
1299 2021-01-04               267.054
1300 2021-01-05               269.195
1301 2021-01-06               271.696
1302 2021-01-07               273.003

[1303 rows x 2 columns]
Date                    datetime64[ns]
Consumer_Price_Index           float64
dtype: object


In [6]:
#Creating first filter for inflation table, filtering for range between 2005 and 2020
first_filt_infl_df=infl_df.loc[(infl_df['Date']>='2005-01-01') & (infl_df['Date']<'2021-01-01')]
first_filt_infl_df.head()

,Date,Consumer_Price_Index
1104,2005-01-01,190.7
1105,2005-01-02,191.8
1106,2005-01-03,193.3
1107,2005-01-04,194.6
1108,2005-01-05,194.4


In [7]:
#Creating first filter for gas table, filtering for range between 2005 and 2020
first_filt_gas_df=gas_df.loc[(gas_df['Date']>='2005-01-01') & (gas_df['Date']<'2021-01-01')]
first_filt_gas_df.head()

,Date,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices
522,2005-01-03,1.824,1.788,1.898,1.957
523,2005-01-10,1.837,1.813,1.887,1.934
524,2005-01-17,1.863,1.843,1.902,1.952
525,2005-01-24,1.896,1.880,1.929,1.959
526,2005-01-31,1.953,1.936,1.988,1.992


In [8]:
#Filtering the Inflation one to put on a weekly rate to match with the gas table (Mondays)
infl_fin_flt_df = first_filt_infl_df.loc[first_filt_infl_df['Date'].dt.weekday == 0]
infl_fin_flt_df.head()

,Date,Consumer_Price_Index
1106,2005-01-03,193.300
1113,2005-01-10,199.200
1117,2006-01-02,198.700
1124,2006-01-09,202.900
1128,2007-01-01,202.416


In [9]:
#Merging the two tables using the dates as an inner join.
gas_inflation_merged_df=pd.merge(infl_fin_flt_df,first_filt_gas_df,how='inner', on='Date')
gas_inflation_merged_df.head()

,Date,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices
0,2005-01-03,193.300,1.824,1.788,1.898,1.957
1,2005-01-10,199.200,1.837,1.813,1.887,1.934
2,2006-01-02,198.700,2.281,2.277,2.290,2.442
3,2006-01-09,202.900,2.371,2.363,2.388,2.485
4,2007-01-01,202.416,2.382,2.340,2.465,2.580


In [10]:
#Droping the rows with no values, to remove the ones that do not match between gas and inflation
gas_inflation_merged__dropna_df=gas_inflation_merged_df.dropna()

gas_inflation_merged__dropna_df

,Date,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices
0,2005-01-03,193.300,1.824,1.788,1.898,1.957
1,2005-01-10,199.200,1.837,1.813,1.887,1.934
2,2006-01-02,198.700,2.281,2.277,2.290,2.442
3,2006-01-09,202.900,2.371,2.363,2.388,2.485
4,2007-01-01,202.416,2.382,2.340,2.465,2.580
5,2007-01-08,207.917,2.354,2.304,2.458,2.537
6,2008-01-07,219.964,3.159,3.135,3.208,3.376
7,2009-01-05,213.856,1.737,1.720,1.772,2.291
8,2009-01-12,215.949,1.835,1.820,1.866,2.314
9,2010-01-04,218.009,2.718,2.677,2.802,2.797


In [11]:
#Creating a new column that displays only the date year, with the intention to groupby and drop later dates in that year.
gas_inflation_merged__dropna_df['Year'] = gas_inflation_merged__dropna_df['Date'].dt.year
gas_inflation_merged__dropna_df

,Date,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices,Year
0,2005-01-03,193.300,1.824,1.788,1.898,1.957,2005
1,2005-01-10,199.200,1.837,1.813,1.887,1.934,2005
2,2006-01-02,198.700,2.281,2.277,2.290,2.442,2006
3,2006-01-09,202.900,2.371,2.363,2.388,2.485,2006
4,2007-01-01,202.416,2.382,2.340,2.465,2.580,2007
5,2007-01-08,207.917,2.354,2.304,2.458,2.537,2007
6,2008-01-07,219.964,3.159,3.135,3.208,3.376,2008
7,2009-01-05,213.856,1.737,1.720,1.772,2.291,2009
8,2009-01-12,215.949,1.835,1.820,1.866,2.314,2009
9,2010-01-04,218.009,2.718,2.677,2.802,2.797,2010


In [12]:
#Groupby the year, and the minimum date in the 'Date' column, and generating a new dataframe to merge with the present one.
first_day = gas_inflation_merged__dropna_df.groupby('Year')['Date'].min()

In [13]:
first_day

Year
2005   2005-01-03
2006   2006-01-02
2007   2007-01-01
2008   2008-01-07
2009   2009-01-05
2010   2010-01-04
2011   2011-01-03
2012   2012-01-02
2013   2013-01-07
2014   2014-01-06
2015   2015-01-05
2016   2016-01-04
2017   2017-01-02
2018   2018-01-01
2019   2019-01-07
2020   2020-01-06
Name: Date, dtype: datetime64[ns]

In [14]:
#Merging the two dataframes to remove later dates and only leave the first mondays in the respective years.
final_attempt_df=pd.merge(first_day,gas_inflation_merged_df,how='inner', on='Date')

In [15]:
final_attempt_df.head()

,Date,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices
0,2005-01-03,193.300,1.824,1.788,1.898,1.957
1,2006-01-02,198.700,2.281,2.277,2.290,2.442
2,2007-01-01,202.416,2.382,2.340,2.465,2.580
3,2008-01-07,219.964,3.159,3.135,3.208,3.376
4,2009-01-05,213.856,1.737,1.720,1.772,2.291


In [16]:
#Droping null values from the dataframe
final_gas_infl_df = final_attempt_df.dropna()
final_gas_infl_df

,Date,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices
0,2005-01-03,193.300,1.824,1.788,1.898,1.957
1,2006-01-02,198.700,2.281,2.277,2.290,2.442
2,2007-01-01,202.416,2.382,2.340,2.465,2.580
3,2008-01-07,219.964,3.159,3.135,3.208,3.376
4,2009-01-05,213.856,1.737,1.720,1.772,2.291
5,2010-01-04,218.009,2.718,2.677,2.802,2.797
6,2011-01-03,223.467,3.124,3.086,3.201,3.331
7,2012-01-02,227.663,3.358,3.310,3.454,3.783
8,2013-01-07,233.596,3.373,3.304,3.512,3.911
9,2014-01-06,238.343,3.411,3.340,3.557,3.910


In [17]:
#Generating new column to display percentage change to Consumer Price Index column
final_gas_infl_df['CPI_Change']=final_gas_infl_df['Consumer_Price_Index'].pct_change(periods=1)
final_gas_infl_df['CPI_Change']

0          NaN
1     0.027936
2     0.018702
3     0.086693
4    -0.027768
5     0.019420
6     0.025036
7     0.018777
8     0.026060
9     0.020321
10   -0.002257
11    0.006123
12    0.018148
13    0.017504
14    0.035116
15    0.004778
Name: CPI_Change, dtype: float64

In [18]:
#Setting Date as index
final_gas_infl_df.set_index("Date")

,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices,CPI_Change
Date,,,,,,
2005-01-03,193.300,1.824,1.788,1.898,1.957,NaN
2006-01-02,198.700,2.281,2.277,2.290,2.442,0.027936
2007-01-01,202.416,2.382,2.340,2.465,2.580,0.018702
2008-01-07,219.964,3.159,3.135,3.208,3.376,0.086693
2009-01-05,213.856,1.737,1.720,1.772,2.291,-0.027768
2010-01-04,218.009,2.718,2.677,2.802,2.797,0.019420
2011-01-03,223.467,3.124,3.086,3.201,3.331,0.025036
2012-01-02,227.663,3.358,3.310,3.454,3.783,0.018777
2013-01-07,233.596,3.373,3.304,3.512,3.911,0.026060


In [19]:
#New column to display percentage change to average of all grades gas prices
final_gas_infl_df['All_Grades_Prices_avg_change']=final_gas_infl_df['All_Grades_Prices_avg'].pct_change(periods=1)
final_gas_infl_df['All_Grades_Prices_avg_change']

0          NaN
1     0.250548
2     0.044279
3     0.326196
4    -0.450142
5     0.564767
6     0.149375
7     0.074904
8     0.004467
9     0.011266
10   -0.323366
11   -0.074957
12    0.163934
13    0.061167
14   -0.116799
15    0.144268
Name: All_Grades_Prices_avg_change, dtype: float64

In [20]:
final_gas_infl_df['All_Grades_Conventional_Prices_avg_change']=final_gas_infl_df['All_Grades_Conventional_Prices_avg'].pct_change(periods=1)
final_gas_infl_df['All_Grades_Conventional_Prices_avg_change']

0          NaN
1     0.273490
2     0.027668
3     0.339744
4    -0.451356
5     0.556395
6     0.152783
7     0.072586
8    -0.001813
9     0.010896
10   -0.332934
11   -0.090215
12    0.198323
13    0.051462
14   -0.138998
15    0.164620
Name: All_Grades_Conventional_Prices_avg_change, dtype: float64

In [21]:
final_gas_infl_df['All_Grades_Reformulated_Prices_avg_change']=final_gas_infl_df['All_Grades_Reformulated_Prices_avg'].pct_change(periods=1)
final_gas_infl_df['All_Grades_Reformulated_Prices_avg_change']

0          NaN
1     0.206533
2     0.076419
3     0.301420
4    -0.447631
5     0.581264
6     0.142398
7     0.079038
8     0.016792
9     0.012813
10   -0.305032
11   -0.047735
12    0.104928
13    0.078047
14   -0.080599
15    0.113266
Name: All_Grades_Reformulated_Prices_avg_change, dtype: float64

In [22]:
final_gas_infl_df.head()

,Date,Consumer_Price_Index,All_Grades_Prices_avg,All_Grades_Conventional_Prices_avg,All_Grades_Reformulated_Prices_avg,No2_Diesel_Prices,CPI_Change,All_Grades_Prices_avg_change,All_Grades_Conventional_Prices_avg_change,All_Grades_Reformulated_Prices_avg_change
0,2005-01-03,193.300,1.824,1.788,1.898,1.957,NaN,NaN,NaN,NaN
1,2006-01-02,198.700,2.281,2.277,2.290,2.442,0.027936,0.250548,0.273490,0.206533
2,2007-01-01,202.416,2.382,2.340,2.465,2.580,0.018702,0.044279,0.027668,0.076419
3,2008-01-07,219.964,3.159,3.135,3.208,3.376,0.086693,0.326196,0.339744,0.301420
4,2009-01-05,213.856,1.737,1.720,1.772,2.291,-0.027768,-0.450142,-0.451356,-0.447631


In [23]:
#Exporting final dataframe to csv file.
final_gas_infl_df.to_csv(r'cleaning_results\gas_cpi.csv',index=False)